# Extract the data from its csv file

In [11]:
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv

In [12]:
class USDACSVExplorer:
    """
    Explore and understand the structure of CSV files
    """
    
    def __init__ (self, data_path:str):
        self.data_path = Path(data_path)
        
    def list_csv_files(self):
        """
        List all CSV files in the data directory
        """
        csv_files = list(self.data_path.glob("*.csv"))
        print(f"Found {len(csv_files)} CSV files.")
        for file in csv_files:
            size_mb = file.stat().st_size / (1024 * 1024)
            print(f"  {file.name:<25} ({size_mb:.2f} MB)")
        return csv_files

    def explore_csv_structures(self, filename: str, sample_rows: int =5):
        """
        Explore the structure of a CSV file
        """
        file_path = self.data_path/filename
        
        if not file_path.exists():
            print(f"File {filename} does not exist in {self.data_path}.")
            return None
        
        print(f"\n{'='*60}")
        print(f"EXPLORING: {filename}")
        print(f"{'='*60}\n")
        
        try:
            
            #Read CSV file
            df = pd.read_csv(file_path)
            
            #Basic Info
            
            print(f"Shape: {df.shape} (rows,columns)")
            print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
            
             # Column info
            print(f"\nColumns ({len(df.columns)}):")
            for i, col in enumerate(df.columns):
                dtype = df[col].dtype
                null_count = df[col].isnull().sum()
                null_pct = (null_count / len(df)) * 100
                print(f"  {i:2d}. {col:<25} | {dtype:<10} | {null_count:>6} nulls ({null_pct:4.1f}%)")
            
            # Sample data
            print(f"\nFirst {sample_rows} rows:")
            print(df.head(sample_rows).to_string())
            
            # Unique values for small categorical columns
            for col in df.columns:
                if df[col].dtype == 'object' and df[col].nunique() < 20:
                    print(f"\nUnique values in '{col}':")
                    print(df[col].value_counts().head(10).to_string())
                    
            return df
        except Exception as e:
            print(f"Error reading {filename}: {e}")
            return None
        
    def compare_food_files(self):
        """Compare food.csv and foundation_food.csv if both exist"""
        food_files = []
        
        # Check for different possible file names
        possible_names = ['food.csv', 'foundation_food.csv']
        for name in possible_names:
            if (self.data_path / name).exists():
                food_files.append(name)
        
        print(f"\nFound food-related files: {food_files}")
        
        if len(food_files) >= 2:
            print("\nComparing food files...")
            
            # Load both files
            dfs = {}
            for file in food_files:
                dfs[file] = pd.read_csv(self.data_path / file)
            
            # Compare structures
            print("\nFile comparison:")
            for file, df in dfs.items():
                print(f"\n{file}:")
                print(f"  Rows: {len(df):,}")
                print(f"  Columns: {list(df.columns)}")
                
                # Check for key identifying columns
                if 'fdc_id' in df.columns:
                    print(f"  FDC ID range: {df['fdc_id'].min()} to {df['fdc_id'].max()}")
                if 'data_type' in df.columns:
                    print(f"  Data types: {df['data_type'].value_counts().to_dict()}")
                if 'description' in df.columns:
                    print(f"  Sample descriptions: {df['description'].head(3).tolist()}")
            
            return dfs
        
        return None
    
    def find_foundation_foods(self):
        """Find how foundation foods are identified in the data"""
        print("\nLooking for foundation foods...")
        
        # Check food.csv first
        food_path = self.data_path / 'food.csv'
        print(f"Checking for food.csv in {self.data_path}...")
        if food_path.exists():
            food_df = pd.read_csv(food_path)
            print(f"\nfood.csv structure:")
            print(f"  Shape: {food_df.shape}")
            print(f"  Columns: {food_df.columns.tolist()}")
            
            # Check for data_type column
            if 'data_type' in food_df.columns:
                print(f"  Data types found: {food_df['data_type'].value_counts()}")
            
            # Check for foundation-related keywords in descriptions
            if 'description' in food_df.columns:
                foundation_keywords = food_df['description'].str.contains('foundation', case=False, na=False)
                print(f"  Foods with 'foundation' in description: {foundation_keywords.sum()}")
        
        # Check if there's a separate foundation_food.csv
        foundation_path = self.data_path / 'foundation_food.csv'
        if foundation_path.exists():
            foundation_df = pd.read_csv(foundation_path)
            print(f"\nfoundation_food.csv structure:")
            print(f"  Shape: {foundation_df.shape}")
            print(f"  Columns: {foundation_df.columns.tolist()}")
            print(f"  Sample data:")
            print(foundation_df.head(3).to_string())
        
        # Check data_type.csv for foundation food type ID
        data_type_path = self.data_path / 'data_type.csv'
        if data_type_path.exists():
            data_type_df = pd.read_csv(data_type_path)
            print(f"\ndata_type.csv:")
            print(data_type_df.to_string())
    
    def quick_overview(self):
        """Get a quick overview of all CSV files"""
        print("USDA FoodData Central - Quick Overview")
        print("=" * 50)
        
        # List files
        csv_files = self.list_csv_files()
        
        # Key file analysis
        key_files = ['food.csv', 'foundation_food.csv', 'nutrient.csv', 'food_nutrient.csv']
        
        for file in key_files:
            if (self.data_path / file).exists():
                print(f"\n{file.upper()}:")
                df = pd.read_csv(self.data_path / file)
                print(f"  Rows: {len(df):,}")
                print(f"  Columns: {', '.join(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
                
                # Show sample for small files
                if len(df) < 100:
                    print(f"  Sample data:")
                    print(df.head(3).to_string())
        
        # Find foundation foods
        self.find_foundation_foods()


In [13]:
load_dotenv()
data_path = os.getenv("USDA_DATA_PATH")

explorer = USDACSVExplorer(data_path=data_path)
explorer.quick_overview()

USDA FoodData Central - Quick Overview
Found 34 CSV files.
  acquisition_samples.csv   (0.14 MB)
  agricultural_samples.csv  (0.04 MB)
  branded_food.csv          (896.07 MB)
  fndds_derivation.csv      (0.00 MB)
  fndds_ingredient_nutrient_value.csv (34.61 MB)
  food.csv                  (205.59 MB)
  food_attribute.csv        (128.62 MB)
  food_attribute_type.csv   (0.00 MB)
  food_calorie_conversion_factor.csv (0.14 MB)
  food_category.csv         (0.00 MB)
  food_component.csv        (0.17 MB)
  food_nutrient.csv         (1684.46 MB)
  food_nutrient_conversion_factor.csv (0.01 MB)
  food_nutrient_derivation.csv (0.01 MB)
  food_nutrient_source.csv  (0.00 MB)
  food_portion.csv          (3.21 MB)
  food_protein_conversion_factor.csv (0.00 MB)
  food_update_log_entry.csv (103.62 MB)
  foundation_food.csv       (0.02 MB)
  input_food.csv            (1.89 MB)
  lab_method.csv            (0.01 MB)
  lab_method_code.csv       (0.00 MB)
  lab_method_nutrient.csv   (0.01 MB)
  market_acqui

C:\Users\Usuario\AppData\Local\Temp\ipykernel_29552\2495531935.py:159: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self.data_path / file)


  Rows: 26,805,037
  Columns: id, fdc_id, nutrient_id, amount, data_points...

Looking for foundation foods...
Checking for food.csv in C:\Users\Usuario\Documents\FoodData_Central_csv_2025-04-24\FoodData_Central_csv_2025-04-24...

food.csv structure:
  Shape: (2064912, 5)
  Columns: ['fdc_id', 'data_type', 'description', 'food_category_id', 'publication_date']
  Data types found: data_type
branded_food                1977398
sub_sample_food               62022
sr_legacy_food                 7793
market_acquistion              7215
survey_fndds_food              5432
sample_food                    3717
agricultural_acquisition        810
foundation_food                 411
experimental_food               114
Name: count, dtype: int64
  Foods with 'foundation' in description: 7

foundation_food.csv structure:
  Shape: (340, 3)
  Columns: ['fdc_id', 'NDB_number', 'footnote']
  Sample data:
   fdc_id  NDB_number footnote
0  321358       16158      NaN
1  321360      100147      NaN
2  3216